In [87]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind1 = pd.read_csv("./dataset/wind1.csv")
wind2 = pd.read_csv("./dataset/wind2.csv")
wind3 = pd.read_csv("./dataset/wind3.csv")
wind4 = pd.read_csv("./dataset/wind4.csv")

In [88]:
from torch.optim.lr_scheduler import ExponentialLR

# Hyper-Parameters
sequence_length = 36
input_size = 2
hidden_size = 64
num_layers = 3
output_size = 4
batch_size = 64
num_epochs = 32
learning_rate = 1e-5

In [89]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y,seq_length):
        self.input_dat = torch.tensor(input_dat).float()
        self.y = torch.tensor(y).float()
        self.seq_length = seq_length

    def __getitem__(self, idx):
        if idx >= self.seq_length - 1:
            i_start = idx - self.seq_length + 1
            x = self.input_dat[i_start:(idx-3),:]
            y = self.y[(idx-3):(idx+1)]
        elif idx>=3:
            padding = self.input_dat[0].repeat(self.seq_length-idx-1,1)
            x = self.input_dat[0:(idx-3),:]
            x = torch.cat((padding,x),0)
            y = self.y[(idx-3):(idx+1)]
        else:
            x = self.input_dat[0].repeat(self.seq_length-4,1)
            padding_y = self.y[0].repeat(3-idx)
            y = self.y[0:(idx+1)]
            y = torch.cat((padding_y,y),0)
        return x, y
    
    def __len__(self):
        return self.input_dat.shape[0]

Using two strategies,

1. Test using MA data and Test relative error using MA data
2. Test using original data and test re using original data

Later, I'll figure out how to test using original data

Split the 4th dataset to test set and the first 3 dataset as training set.

In [90]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)
test_data1 = mydata(test,test[:,1],sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [91]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

lstm_ma = RNN(input_size, hidden_size, num_layers, output_size).to(device)
lstm_o = RNN(input_size,hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op_ma = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
op_o = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
scheduler_ma = ExponentialLR(op_ma,gamma=0.9)
scheduler_o = ExponentialLR(op_o,gamma=0.9)

In [92]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_ma.step()

0.11506940424442291
0.08616837114095688
0.04883268475532532
0.11819012463092804
0.07497565448284149
0.04663965851068497
0.12317994982004166
0.08749169111251831
0.045602474361658096
0.11659349501132965
0.07471343874931335
0.04418050870299339
0.1252126544713974
0.08391319960355759
0.04215799644589424
0.1365242749452591
0.08576434850692749
0.04119434580206871
0.11839771270751953
0.07265973836183548
0.04051946848630905
0.12625408172607422
0.06651438772678375
0.038536012172698975
0.11431173980236053
0.07074868679046631
0.03774336725473404
0.14962518215179443
0.0666569396853447
0.03605664148926735
0.14610067009925842
0.08357097208499908
0.03567812219262123
0.12921138107776642
0.06921297311782837
0.03445029631257057
0.09498874843120575
0.0701906681060791
0.03517436236143112
0.1192149743437767
0.06775183230638504
0.03388034552335739
0.09327922761440277
0.06597660481929779
0.033656228333711624
0.11588398367166519
0.06847784668207169
0.03366796672344208
0.11009712517261505
0.0631783977150917
0.0

In [93]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1123)
0.23499589288329475
0.979828417301178
0.14867109805345535
0.5052631578947369


In [94]:
# Evaluation using ma data
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat1:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0640)
0.22141658261156918
0.567604660987854
0.20742802321910858
0.30701754385964913


If just input the original data, let's see what will happen.

In [95]:
# Training Sets
train1 = np.array(wind1[["GridPower","WindSpd"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPower","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPower","WindSpd"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [96]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_o.step()

0.12683239579200745
0.09166690707206726
0.030759423971176147
0.1118517816066742
0.07797396183013916
0.031212294474244118
0.15039777755737305
0.08383212983608246
0.029711220413446426
0.13405296206474304
0.08904244005680084
0.02995527721941471
0.15551041066646576
0.08792262524366379
0.029293227940797806
0.1256210058927536
0.09017501026391983
0.028934204950928688
0.12162861973047256
0.08672021329402924
0.029002349823713303
0.13259896636009216
0.08043123036623001
0.02854018844664097
0.1169380396604538
0.08782452344894409
0.027633359655737877
0.14876282215118408
0.0848732590675354
0.02748604118824005
0.10541661828756332
0.08768942952156067
0.028077062219381332
0.12498548626899719
0.0815960243344307
0.02758021280169487
0.13250288367271423
0.09021377563476562
0.027633799239993095
0.13884590566158295
0.0887162908911705
0.02710086852312088
0.1288071721792221
0.0899745300412178
0.02767215296626091
0.14316865801811218
0.09367043524980545
0.02682441659271717
0.11877445876598358
0.08989989757537842

In [97]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1064)
0.22825231062607806
0.9593997597694397
0.1467757448554039
0.5140350877192983
